In [75]:
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
#from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D,LSTM,MaxPool1D,Dense,Embedding,Input,InputLayer,Flatten,UpSampling1D,Lambda,Reshape,BatchNormalization,ZeroPadding1D
from keras.models import Model
from keras.engine.sequential import Sequential
import numpy as np
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from wordcloud import WordCloud
from sklearn import svm
from sklearn.utils import shuffle
import mimetypes as mim
from ast import literal_eval
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
# Adding new types t
mim.add_type(ext='.go',type='text')
mim.add_type(ext='.js',type='text')
mim.add_type(ext='.json',type='text')

In [2]:
#import keras.backend as K
#from keras.layers import Conv2DTranspose

# Data Preparation and Cleaning

In [76]:
# Loading Data 
root ='data/'
repoNamesList=[]
paths=[]
pathNamesList=[]
for pathName in os.walk(root):
        paths+=[pathName]


In [64]:
print(paths[0][1])

['Automotive', 'Banking', 'Education', 'Energy', 'FinTech', 'Food_And_beverages', 'Healthcare', 'Insurance', 'Law', 'media', 'Retail', 'Technology', 'transportation']


In [5]:
#pathsWithFile=paths[2::]
#type(pathsWithFile)
#print(pathsWithFile)
#paths[0][1].remove(paths[0][1][3])

[('data/Banking', [], ['Akka-Content-Based-Router', 'alldaysecops', 'apiconnect', 'awesome-ddd', 'awesome-devsecops_ru', 'awesome-k8s-lessons-learned', 'awesome-scalability', 'awesome-stars', 'banking-microservices-tutorial', 'BankingInTheCloud-WorkshopSetup', 'bian', 'cascadia', 'ClientRating', 'cloudtestdrive', 'code-robots', 'companies', 'conversation-with-linuxone-using-watson-microservices', 'customers-accounts-and-money-transfers', 'DeathStarBench', 'DeepDetectMessager', 'devopshandbook_endnotes', 'digitalbanking-accountsservice', 'digitalbanking-cardsservice', 'digitalbanking-investmentsservice', 'digitalbanking-loansservice', 'digitalbanking-microservice', 'digitalbanking-txfrjava', 'eventuate-client-spring-cloud-stream', 'fuse-financial-cicd', 'go_read', 'gocn_news_set', 'Hands-On-Design-Patterns-with-C-and-.NET-Core', 'ICp-banking-microservices', 'innovate-digital-bank', 'java-best-practices', 'java-technology-for-beginners', 'Lagom-object-storage', 'Learn-Kotlin-Programming'

In [65]:
labels=paths[0][1]
#labels=np.array(labels)
#labels.reshape(-1,1)
print(labels)

['Automotive', 'Banking', 'Education', 'Energy', 'FinTech', 'Food_And_beverages', 'Healthcare', 'Insurance', 'Law', 'media', 'Retail', 'Technology', 'transportation']


In [77]:
def get_all_files(dirName):
    fullpath =''
    allfiles = ''
    #print(dirName)
    if(os.path.isdir(dirName)):
         Dirs=os.listdir(dirName)
    else:
        allfiles = allfiles+fullpath
        Dirs =[]   
    for entry in  Dirs:
        fullpath=os.path.join(dirName,entry)
        #print(fullpath)
        if os.path.isdir(fullpath):
                allfiles = allfiles+get_all_files(fullpath)
        else:
                allfiles = allfiles+fullpath
        allfiles+=' '
    return allfiles 
  

In [16]:
#print(get_all_files('/Users/gleschen/Downloads/data/Banking'))
enenrgyReposName =paths[1][1]
len(enenrgyReposName)

/Users/gleschen/Downloads/data/Banking/Akka-Content-Based-Router /Users/gleschen/Downloads/data/Banking/alldaysecops /Users/gleschen/Downloads/data/Banking/apiconnect /Users/gleschen/Downloads/data/Banking/awesome-ddd /Users/gleschen/Downloads/data/Banking/awesome-devsecops_ru /Users/gleschen/Downloads/data/Banking/awesome-k8s-lessons-learned /Users/gleschen/Downloads/data/Banking/awesome-scalability /Users/gleschen/Downloads/data/Banking/awesome-stars /Users/gleschen/Downloads/data/Banking/banking-microservices-tutorial /Users/gleschen/Downloads/data/Banking/BankingInTheCloud-WorkshopSetup /Users/gleschen/Downloads/data/Banking/bian /Users/gleschen/Downloads/data/Banking/cascadia /Users/gleschen/Downloads/data/Banking/ClientRating /Users/gleschen/Downloads/data/Banking/cloudtestdrive /Users/gleschen/Downloads/data/Banking/code-robots /Users/gleschen/Downloads/data/Banking/companies /Users/gleschen/Downloads/data/Banking/conversation-with-linuxone-using-watson-microservices /Users/gles

0

In [17]:
# we will count how many class

#reposNameList = []
allFiles=[]
pathsDic = {}
#Y=[]
#YDic={}

for label in labels: 
    #print(label)
    #YDic[label] = []
    pathsDic[label]={}
    for l in os.listdir(path='./data/'+label):
        #Y+=[label]
        #YDic[label] += [l]
        path=get_all_files('./data/'+label+'/'+l)
        if (re.search('README',path) or re.search('README',path) or path.endswith('.'+path.split('.')[-1])):
            pathsDic[label][l] = path
            
        #pathsDic[label][l] =
        #repoDic[label][l] =repoDic[label][l][0].replace('  ','')
        #repoDic[label][l]=repoDic[label][l].split(' ')
        
   

In [78]:
cleanedData=''
root ='data'
for label in repoDic:  
    repoDic2={}
    if(os.path.isdir(os.path.join(root,label))==False):
        os.mkdir(os.path.join(root,label))    
    for repoName in repoDic[label]:
        repoDic2[repoName]={}  
        for paths in repoDic[label][repoName]:
            for path in paths.split():
                 #print(path)   
                typeGuess = mim.guess_type(path)
                if(typeGuess[0] is not None):
                    typeGuess=typeGuess[0].split('/')
                    checkImg=check_img(typeGuess)
                    checkText=check_text(typeGuess)
                    if(checkImg):  
                        #print(fileBytes)
                        fileName=path.split('/')[-1]
                        if(fileName.split('.')[-1] == 'SVG' or  fileName.split('.')[-1] == 'svg'):
                             continue
                        #Extract the text from the image.
                        if(fileName.split('.')[-1]=='png'or fileName.split('.')[-1]=='PNG'):
                            try:
                                imgFile=Image.open(path,'r').convert('RGB')
                            except:
                                continue
                            imgFile.save('ImgToSave.jpg')
                            fileName='ImgToSave.jpg'
                            try:
                                textOfImg=pytesseract.image_to_string(imread(fileName))
                                textOfImg=data_clean(textOfImg)
                            except:
                                   continue    
                        else:
                            try:
                                textOfImg=pytesseract.image_to_string(imread(path))
                                textOfImg=data_clean(textOfImg)
                            except:
                                   continue 
                        imgFile.close()        
                        repoDic2[repoName][path.split('/')[-1]]=textOfImg
                       
                    elif(checkText):
                        try:
                            file=open(path,'r')
                            lines=file.readlines()
                        except:
                            print(path)
                            continue   
                        cleanedData=data_clean(lines)         
                        repoDic2[repoName][path.split('/')[-1]]=cleanedData
                        
                elif(re.search('description',path)or re.search('README',path) or re.search('readme',path)):
                    try:
                        file=open(path,'r')      
                        lines=file.readlines()
                    except:
                        print(path)
                        continue
                    cleanedData=data_clean(lines)
                    repoDic2[repoName][path.split('/')[-1]]=cleanedData
        repoFile = open(os.path.join(root,os.path.join(label,repoName)),'w')
        repoFile.write(str(repoDic2))
        repoFile.close()
        repoDic2={}

NameError: name 'repoDic' is not defined

In [31]:
def data_clean(lines):
    print(lines)
    cummulateString=''
    checkAlpha = 0
    for line in lines:
        line=line.replace('\\n',' ') 
        line=line.replace('.',' ')
        line=line.replace(':',' ')
        line=line.replace(';',' ')
        line=line.replace('(',' ')
        for char in line: 
            #Exclude the non alphabitical data.
            #print(char)
            if(char.isalpha() or char.isupper()):
                char = char.lower()
                cummulateString+=char
                checkAlpha=1
                #print(char)
            elif(char.isspace()):
                if(checkAlpha == 1):
                        cummulateString+=' '
                        checkAlpha=0 
                else:
                        cummulateString+=''
               
            else:
                cummulateString+=''
    cummulateString+=' ' 
    return cummulateString

In [32]:
file = open('/Users/gleschen/Downloads/test_raw_file.txt','r')
print(data_clean(file.readlines()))

['*\\n *  or in the "license" file accompanying this file. This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES *\\{\'iot-device-simulator\': {\'desc\': \'The IoT Device Simulator solution is a Graphical User Interface (GUI) based engine designed to enable customers to get started quickly assessing AWS IoT services without an existing pool of devices. The IoT Device Simulator helps effortlessly create and simulate thousands of connected devices that are defined by the customer.\', \'app.js\': \'/*********************************************************************************************************************\\n *  Copyright 2016 Amazon.com, Inc. or its affiliates. All Rights Reserved.                                           *\\n *                                                                                                                    *\\n *  Licensed under the Amazon Software License (the "License"). You may not use this file except in compliance        *\\n 

In [33]:
def check_text(typeGuess):
    if(re.search(typeGuess[0],'text')):
         return True
    else:
         return False

In [44]:
typeGuess = mim.guess_type ('/Users/gleschen/Downloads/test_raw_file.txt') 
print(typeGuess[0].split('/')[0])

text


In [115]:
def check_img(typeGuess):
    if(re.search(typeGuess[0],'image')):
         return True
    else:
         return False

In [74]:
'''
repoDic2={}    
root ='data'
for label in repoDic:   
    if(os.path.isdir(os.path.join(root,label))==False):
        os.mkdir(os.path.join(root,label))
    for repoName in repoDic[label]:
        repoDic2[repoName]={}  
        if(os.path.isdir(label+"/"+repoName)):
              os.mkdir(os.path.join(label,repoName))
        for path in repoDic[label][repoName]:
            path=path.split()
            print(path)
'''            

'\nrepoDic2={}    \nroot =\'newData\'\nfor label in repoDic:   \n    if(os.path.isdir(os.path.join(root,label))==False):\n        os.mkdir(os.path.join(root,label))\n    for repoName in repoDic[label]:\n        repoDic2[repoName]={}  \n        if(os.path.isdir(label+"/"+repoName)):\n              os.mkdir(os.path.join(label,repoName))\n        for path in repoDic[label][repoName]:\n            path=path.split()\n            print(path)\n'

In [79]:
#Define variables
Y=[]
reposNameList=[]
reposContentsList=[]
contentMerged = ''

In [82]:
# Merge all files of each microservice repo in one string
root = 'data'
for label in labels: 
    
    for repoName in os.listdir(path='./data/'+label):
        Y+=[label]
        repoNamesList+=[repoName]
        filesContent=open('./data/'+label+'/'+repoName,'r')
        try:
            contentDic=literal_eval(filesContent.read())
        except:
            continue
        print(contentDic)
        contentMerged = ''
        for filesKey in contentDic[repoName]:
                    contentMerged+=repoName
                    contentMerged+=' ' 
                    contentMerged+= contentDic[repoName][filesKey]
        reposContentsList+=[contentMerged]        
        filesContent.close() 
        #path=get_all_files('./data/'+label+'/'+l)

{'awesome-devops-br': {'README.md': ' awesome devops br lista de recursos  links  livros  q amp a  discutidos no canal do telegram devops br  https   t me devopsbr    table of contents   awesome devops br   awesome devops br    por onde começar   por onde começar    ferramentas   ferramentas    livros   livros    q amp a   q  a    bookmarks   bookmarks    segurança   segurança    blogs técnicos de grandes empresas   blogs técnicos de grandes empresas    por onde começar   gutocarvalho  http   gutocarvalho net blog            por onde iniciar os estudos sobre devops    por onde iniciar os estudos sobre devops    origens  http   gutocarvalho net blog            origens da cultura devops    origens da cultura devops   perguntas comuns  http   gutocarvalho net octopress            perguntas pontuais devops     quora  https   www quora com what are the best resources for learning about devops   what are the best resources for learning about devops    opsschool  http   www opsschool org en l

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [81]:
len(repoNamesList)
len(Y)
len(reposContentsList)

142

In [145]:
'''
cummulateString=''
fileStringsLs = []
repoNameList = []
indForMerge = 0
#countSpace = 0
pathsWithFile[0][0]
cnt=0
 for indx in range(0,len(pathsWithFile)):
                if(pathsWithFile[indx][-1] == [] or pathsWithFile[indx][-2] == []):
                    #print(1)
                    continue
                else:
                  if(len(pathsWithFile[indx][0].split('/'))>2):
                    if(len(repoNameList)>=1 and repoNameList[-1]!=pathsWithFile[indx][0].split('/')[2]):
                        print(repoNameList[-1],pathsWithFile[indx][0].split('/')[2])
                        #print(pathsWithFile[indx][0].split('/')[2])
                        cnt+=1
                        print(cnt)
                        fileStringsLs+=[cummulateString]
                        cummulateString = ''
                    for fileName in pathsWithFile[indx][-1]:
                        #print(fileName)
                        if re.search('README',fileName)or re.match('description',fileName):
                            #print(fileName)
                            #print(fileName)
                            try:
                                 file=open(pathsWithFile[indx][0]+'/'+fileName,'r') 
                                 lines=file.readlines()
                            except:
                                  continue
                                
                            
                            for line in lines:
                                #print(newLine)
                                for char in line: 
                                    # Exclude the non alphabitical data.
                                    if(char.isalpha()):
                                         cummulateString+=char
                                    else:    
                                         cummulateString+=' '     
                                    
                                cummulateString+=' '
                            #print(cummulateString)
                            file.close() 
                    cummulateString+=' '
                    #print(pathsWithFile[indx][0])
                    if(len(pathsWithFile[indx][0].split('/'))>2):
                         repoNameList+=[pathsWithFile[indx][0].split('/')[2]]
                    
         fileStringsLs+=[cummulateString]
                       
'''

"\ncummulateString=''\nfileStringsLs = []\nrepoNameList = []\nindForMerge = 0\n#countSpace = 0\npathsWithFile[0][0]\ncnt=0\n for indx in range(0,len(pathsWithFile)):\n                if(pathsWithFile[indx][-1] == [] or pathsWithFile[indx][-2] == []):\n                    #print(1)\n                    continue\n                else:\n                  if(len(pathsWithFile[indx][0].split('/'))>2):\n                    if(len(repoNameList)>=1 and repoNameList[-1]!=pathsWithFile[indx][0].split('/')[2]):\n                        print(repoNameList[-1],pathsWithFile[indx][0].split('/')[2])\n                        #print(pathsWithFile[indx][0].split('/')[2])\n                        cnt+=1\n                        print(cnt)\n                        fileStringsLs+=[cummulateString]\n                        cummulateString = ''\n                    for fileName in pathsWithFile[indx][-1]:\n                        #print(fileName)\n                        if re.search('README',fileName)or re.

In [70]:
len(fileStringsLs)

NameError: name 'fileStringsLs' is not defined

In [71]:
fileStringsLs[0]

NameError: name 'fileStringsLs' is not defined

In [9]:
# Remove space .
for indx in range(0,len(reposContentsList)):
     reposContentsList[indx]=' '.join(reposContentsList[indx].split())

In [10]:
# Clean data from the extensions.
ext=['ymal','png','jpg','jpeg','js','src','java','xml','com','jar','html','yml','main','test']
for indx in range(0,len(reposContentsList)):
     for indx2 in range(0,len(ext)):
          reposContentsList[indx]=reposContentsList[indx].replace(ext[indx2],"")
     

In [61]:
stem=PorterStemmer()
stemmingWordsLs=[]
Lemmatizer = WordNetLemmatizer()
lemmatizWordsLs=[]
fileStringsLs2= []
fileStringLs3 = []
# Remove Duplications and stemmer
for indx in range(0,len(fileStringsLs)):
        fileStringsLs[indx] = list(dict.fromkeys(fileStringsLs[indx].split(' ')))
        for word in fileStringsLs[indx]:
                stemmingWordsLs+=[stem.stem(word)]
                lemmatizWordsLs+=[Lemmatizer.lemmatize(word)]
        fileStringsLs[indx]= stemmingWordsLs   
        fileStringsLs2+=lemmatizWordsLs
        #print(fileStringsLs2)
        stemmingWordsLs=[]
        lemmatizWordsLs=[]
        fileStringsLs[indx]=' '.join(fileStringsLs[indx])
        fileStringLs3 += [' '.join(fileStringsLs2)]
        fileStringsLs2=[]

NameError: name 'fileStringsLs' is not defined

In [ ]:
len(fileStringsLs)

# Data Encoding And Training

In [72]:
# Encodeing the text and labels
vectorizer = TfidfVectorizer(analyzer='word',stop_words='english')
X_encoded = vectorizer.fit_transform(reposContentsList)
labelEnc =LabelEncoder()
Y_encoded=labelEnc.fit_transform(Y)
#shuffle randomly
X_encoded,Y_encoded=shuffle(X_encoded,Y_encoded)
print(X_encoded \tY_encoded)

ValueError: Found input variables with inconsistent numbers of samples: [142, 143]

In [ ]:
xTrain,xTest,yTrain,yTest=train_test_split(X_encoded,Y_encoded,test_size=0.33)

In [ ]:
'''
one=OneHotEncoder()
xTrain,xTest=train_test_split(X,test_size=0.33)
xtrain=one.fit_transform(xTrain.toarray())
xtest= one.fit_transform(xTest.toarray())
#label=LabelEncoder()
#label.fit_transform(xTrain)
'''

In [ ]:
'''
filtersize = (3,3)
numUnits = 10 
input_shape = xtrain.shape
print(input_shape)
model = Sequential()
model.add(Embedding(input_dim=input_shape[0],input_length=input_shape[1],output_dim=10))
#model.add(Conv1D(numUnits,3,padding='same'))
#model.add(MaxPool1D(pool_size=2))
#model.add(Conv1D(numUnits,3))
#model.add(MaxPool1D(pool_size=2))
#model.add(Conv1D(numUnits,3,padding='same'))
model.add(MaxPool1D(pool_size=3))
model.add(Conv1D(numUnits,3,activation='relu',padding='same'))
encodedModel = model
model.add(Lambda(lambda x: K.expand_dims(x, axis=2)))
model.add(Conv2DTranspose(filters=10, kernel_size=(3, 1),activation='relu',padding='same'))
model.add(Lambda(lambda x: K.squeeze(x, axis=2)))
model.add(UpSampling1D(size=3))
model.add(ZeroPadding1D(padding=1))
#model.add(Lambda(lambda x: K.expand_dims(x, axis=2)))
#model.add(Conv2DTranspose(filters=10, kernel_size=(3, 1),padding='same'))
#model.add(Lambda(lambda x: K.squeeze(x, axis=2)))
#model.add(UpSampling1D(size=2))
#model.add(Lambda(lambda x: K.expand_dims(x, axis=2)))
#model.add(Conv2DTranspose(filters=10, kernel_size=(3, 1)))
#model.add(Lambda(lambda x: K.squeeze(x, axis=2)))
#model.add(UpSampling1D(size=2))
#model.add(Lambda(lambda x: K.expand_dims(x, axis=2)))
#model.add(Conv2DTranspose(filters=10, kernel_size=(3, 1),padding='same'))
#model.add(Lambda(lambda x: K.squeeze(x, axis=2)))
model.add(Dense(1))
model.add(Lambda(lambda x: K.squeeze(x, axis=2)))
#model.add(Lambda(lambda x: K.squeeze(x, axis=1)))
#print(list(model.layers[-1].output_shape))
#newdim = tuple([x for x in list(model.layers[-1].output_shape) if x !=10 and x is not None])
#print(newdim)
#model.add(Reshape((0,newdim[0])))
model.compile(loss='mse',optimizer='Adam')
model.summary()
model.fit(xtrain,xtrain,epochs=20)
'''

In [ ]:
'''
autoencoder = Sequential()
autoencoder.add(Dense(xtrain.shape[1],input_shape=xtrain.shape))
autoencoder.add(Dense(50))
autoencoder.add(Dense(50))
autoencoder.add(Dense(xtrain.shape[1]))
autoencoder.compile(loss='mse',optimizer='sgd')
autoencoder.summary()
autoencoder.fit(xtrain,xtrain,epochs=10)
# to drop all decoded layers to get the encoded layers.
# this step will use it to get encoded data from CNN it's called feature extraction.
for layer in reversed(encodedModel.layers):
     if(re.search('Conv1D',str(layer))):
        break
     encodedModel.pop()
     # add flatten layer to put them in 2 dimesion shape of the previous layer to one dimension.
     #encodedModel.layers
     encodedModel.add(Flatten())
     predictionTs=model.predict(xtest)
     #predictionTr=model.predict(xtrain)
     #encodedData.shape
     #encodedData=encodedModel.predict(xtest)
'''

# KMeans Clustering

In [74]:
true_k =[14]
sse=[]
KMeansModelsLs =[]
for indx in range(0,len(true_k)):
    KMeansModelsLs += [KMeans(n_clusters=true_k[indx],init="k-means++", max_iter=100, n_init=1,random_state=3452)]
    KMeansModelsLs[-1].fit(xtrain)
    sse.append(KMeansModelsLs[-1].inertia_)
    print(sse)
# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(true_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');


NameError: name 'xtrain' is not defined

#  Show results

In [ ]:
# Prediction for training.
termsLs =[]
for KmsModlindx in range(0,len(KMeansModelsLs)):
    print("Kmeans number of cluster",true_k[indx])
    order_centroids = KMeansModelsLs[-1].cluster_centers_.argsort()[:, ::-1]
    #print(order_centroids)
    terms = vectorizer.get_feature_names()
    #print(terms)
    for i in range(true_k[indx]):
        print("Cluster %d:" % i),
        for ind in order_centroids[i,:]:
            #print(ind)
            print(' %s' % terms[ind])
            termsLs +=[terms[ind]]
    print("\n")
    print("Prediction")
 

In [73]:
for KmsModlindx in range(0,len(KMeansModelsLs)):
       prediction = KMeansModelsLs[KmsModlindx].predict(xtest)
print(prediction)

NameError: name 'xtest' is not defined

In [ ]:
# Prediction for Testing.
for KmsModlindx in range(0,len(KMeansModelsLs)):
    print("Kmeans number of cluster",true_k[indx])
    order_centroids = KMeansModelsLs[-1].cluster_centers_.argsort()[:, ::-1]
    #print(order_centroids)
    terms = vectorizer.get_feature_names()
    #print(terms)
    for clusterIndx in range(true_k[indx]):
        print("Cluster %d:" % clusterIndx),
        for ind in order_centroids[clusterIndx,50:60]:
          #print(ind)
          print(' %s' % terms[ind])
          
    print("\n")
    print("Prediction")
    prediction =  KMeansModelsLs[-1].predict(xTest[0:20])
    print(prediction) 

In [ ]:
wordcloud = WordCloud(width = 1000, height = 1000, 
                background_color ='white', 
                min_font_size = 10).generate(' '.join(terms[12])) 


In [ ]:
plt.imshow(wordcloud)

In [ ]:
terms=np.array(termsLs)

In [ ]:
terms=terms.reshape(true_k[0],xtrain.shape[1])

# Classification 

In [ ]:
SVM=svm.SVC(C=1.0,gamma='scale',random_state=42,kernel='poly')
mlp = MLPClassifier(hidden_layer_sizes=(50,))

In [ ]:
SVM.fit(xTrain,yTrain)

In [28]:
mlp.fit(xTrain,yTrain)

MemoryError: Unable to allocate array with shape (6191078, 50) and data type float64

In [ ]:
SVM.score(xTrain,yTrain)*100

In [ ]:
mlp.score(xTrain,yTrain)*100

In [21]:
yPred=SVM.predict(xTest)

In [ ]:
yPred = mlp.predict(xTrain)

In [22]:
accuracy_score(yTest,yPred,normalize=True)*100

9.30232558139535